# 1. 랜덤 포레스트 

## 1) 앙상블 학습 
  - 앙상블 : 정형 데이터를 다루는데 가장 뒤어난 알고리즘 
      - 결정트리를 기반으로 만들어짐 
  -  대다수 정형 데이터에 적합
-  사이킷 런에서 제공하는 정형 데이터의 끝판왕 

## 2) 랜덤포레스트 
- 앙상블 학습 중에서 안정적인 성능을 사용 
- 결정 트리를 랜덤하게 만들어 결정트리의 숲을 만듦 -> 결정 트리의 예측을 사용해 최종 예측을 만듦
### 부트 스트랩 샘플
   - 트리 훈련을 위해 데이터를 랜덤하게 만듦
    - 훈련 데이터를 랜덤하게 추출
    - 중복도 가능 

- 기본적으로 100개의 결정 트리를 훈련 -> 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음 

### 특징 
- 랜덤 샘플 선택 > 과대 적합 방지 
- 검증 세트와 테스트 세트에서 안정적인 성능

# 2. 사이킷런의 랜덤포레스트

## 1) 데이터 불러오기 + 훈련세트 분리

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv("https://bit.ly/wine_csv_data")

data = wine[["alcohol", "sugar", "pH"]]
target = wine["class"]

train_input, test_input, train_target, test_target = train_test_split(data,target,test_size=0.2, random_state=42)

## 2) 교차검증 
- `n_jobs=-1` : 모든 CPU 코어를 사용하여 병렬 처리
- `return_train_score=True` : 훈련 세트에 대한 점수도 반환

In [7]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

### 교차검증
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

### 모델 평가
print("훈련 세트 평가 점:", np.mean(scores["train_score"]))
print("테스트 세트 평가 점:", np.mean(scores["test_score"]))

훈련 세트 평가 점: 0.9973541965122431
테스트 세트 평가 점: 0.8905151032797809


- 훈련 세트 : 0.99  VS 테스트 세트 : 0.89 -> 과대 적합 상태 

### 3) OBB 샘플 

- `oob_score = True` 옵션을 사용하면 OOB 샘플을 이용해서 모델을 평가할 수 있음
- 자체적으로 모델을 평가하는 점수 
- OBB 샘플 : 부트스트랩에 포함되지 않은 샘플 
  -  이 샘플들을 이용해서 검증 세트처럼 사용함 

In [8]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print("OOB 추정 정확도:", rf.oob_score_)

OOB 추정 정확도: 0.8934000384837406


# 3. 엑스트라 트리 

- 랜덤포레스트와 다르게 부트스트랩 샘플을 사용하지 않음 
- 각 결정 트리가 만들어질때 `전체 훈련 세트` 를 사용 
  - 단, 노드를 분할 할 때 `무작위` 로 분할 
  - 랜덤 포레스트보다 빠른 것이 장점 

In [11]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print("훈련 세트 평가 점:", np.mean(scores["train_score"]))
print("테스트 세트 평가 점:", np.mean(scores["test_score"]))

훈련 세트 평가 점: 0.9974503966084433
테스트 세트 평가 점: 0.8887848893166506


# 4. 그레디언트 부스팅 
- 깊이가 얕은 결정 트리를 사용 -> 트리의 오차를 보완 
- 깊이가 3인 결정 트리를 100개 사용
- 과대적합에 강하고 높은 일반화 성능 기대 
- 경사하강법을 사용하여 트리 앙상블에 추가 
        - 손실함수 : 로지스틱 손실함수 

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print("훈련 세트 평가 점:", np.mean(scores["train_score"]))
print("테스트 세트 평가 점:", np.mean(scores["test_score"]))

훈련 세트 평가 점: 0.8881086892152563
테스트 세트 평가 점: 0.8720430147331015


- 학습률을 증가시키고 트리의 개수를 늘림 -> 성능 상승 
- `n_estimators` : 트리의 갯수 
- `learning_rate` : 학습률`

In [13]:
gb= GradientBoostingClassifier(random_state=42, n_estimators=500, learning_rate=0.2)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print("훈련 세트 평가 점:", np.mean(scores["train_score"]))
print("테스트 세트 평가 점:", np.mean(scores["test_score"]))

훈련 세트 평가 점: 0.9464595437171814
테스트 세트 평가 점: 0.8780082549788999


# 5. 히스토그램 기반 그레디언트 부스팅 
  - 그레디언트 부스팅은 트리를 순차적으로 학습하는 방식이기 때문에 훈련 시간이 오래 걸림 
- 속도와 성능을 개선한 것이 히스토그램 기반 그레디언트 부스팅
- 정형 데이터를 다루는 머신러닝 알고리즘 중 인기가 좋음 

### 원리 
- 입력 구간을 256개로 나룸 
- 각 구간에 해당하는 값을 대표하는 값으로 변환

In [14]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42, max_iter=100)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print("훈련 세트 평가 점:", np.mean(scores["train_score"]))
print("테스트 세트 평가 점:", np.mean(scores["test_score"]))

훈련 세트 평가 점: 0.9321723946453317
테스트 세트 평가 점: 0.8801241948619236


### permutation importance 
  - 특성을 하나씩 랜덤하게 섞어 모델의 성능 변화를 찰
- `n_repeats` : 섞는 횟수

In [16]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print("특성 중요도:\n", result.importances_mean)

특성 중요도:
 [0.05969231 0.20238462 0.049     ]


In [17]:
### 최종 성능 확인 
hgb.score(test_input, test_target)
print("테스트 세트 정확도:", hgb.score(test_input, test_target))

테스트 세트 정확도: 0.8723076923076923


# XGBoost

In [22]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method="hist", random_state=42, n_jobs=-1)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print("훈련 세트 평가 점:", np.mean(scores["train_score"]))
print("테스트 세트 평가 점:", np.mean(scores["test_score"]))

훈련 세트 평가 점: 0.9567059184812372
테스트 세트 평가 점: 0.8783915747390243


In [27]:
## LightGBM
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(random_state=42, n_jobs=-1)
scores = cross_validate(lgbm, train_input, train_target, return_train_score=True, n_jobs=-1)
print("훈련 세트 평가 점:", np.mean(scores["train_score"]))
print("테스트 세트 평가 점:", np.mean(scores["test_score"]))

훈련 세트 평가 점: 0.935828414851749
테스트 세트 평가 점: 0.8801251203079884
